# 📊 JPMorgan European Equity Thesis – Data Exploration

This notebook performs **exploratory data analysis (EDA)** for the European equity thesis:

- Download and inspect STOXX 600 and S&P 500 data
- Compute returns, relative performance, and basic risk metrics
- Demonstrate the **validator → cleaner → normalizer → aggregator** pipeline
- Prepare canonical datasets that feed the Streamlit dashboard

> You can run this notebook standalone with just `yfinance` installed, or integrate it with the
> full project by using the connectors and processors under `src/`.

In [ ]:

import os
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

# Plotting (you can switch to Plotly if preferred)
import matplotlib.pyplot as plt

# Market data (fallback)
import yfinance as yf

# Project imports (optional – work even if not yet implemented)
try:
    from config.settings import settings
except Exception as exc:
    print("[WARN] Could not import config.settings.Settings – using defaults.", exc)
    settings = None

# Data processors
validator = cleaner = normalizer = aggregator = None
try:
    from src.data.processors.validator import validator as _validator
    from src.data.processors.cleaner import cleaner as _cleaner
    from src.data.processors.normalizer import normalizer as _normalizer
    from src.data.processors.aggregator import aggregator as _aggregator

    validator = _validator
    cleaner = _cleaner
    normalizer = _normalizer
    aggregator = _aggregator
except Exception as exc:
    print("[WARN] Could not import validator/cleaner/normalizer/aggregator yet:", exc)

plt.style.use("seaborn-v0_8")
pd.options.display.float_format = "{:,.4f}".format

print("✅ Imports complete. pandas:", pd.__version__)


In [ ]:

# ------------------------------------------------------------------
# Configuration
# ------------------------------------------------------------------

# Index tickers (Yahoo Finance)
TICKER_STOXX_600 = "^STOXX"
TICKER_SP500 = "^GSPC"

# Date range for analysis
end_date = datetime.today().date()
start_date = end_date - timedelta(days=5 * 365)  # ~5 years

print(f"Using date range: {start_date} → {end_date}")


In [ ]:

            # ------------------------------------------------------------------
            # Helper: Robust price download
            # ------------------------------------------------------------------

            def fetch_prices_yahoo(ticker: str, start: datetime, end: datetime) -> pd.Series:
                """Fetch adjusted close prices from Yahoo Finance as a Series.

"
                "Args:
"
                "    ticker: Ticker symbol (e.g. '^STOXX').
"
                "    start: Start date.
"
                "    end: End date.

"
                "Returns:
"
                "    Series of adjusted close prices with DateTimeIndex.
"
                "    Empty series if download fails.
"
                """"
"
                try:
"
                "    df = yf.download(ticker, start=start, end=end, progress=False)
"
                "    if df.empty:
"
                "        print(f"[WARN] No data returned for {ticker}")
"
                "        return pd.Series(dtype=float)
"
                "    s = df["Adj Close"].rename(ticker)
"
                "    s.index = pd.to_datetime(s.index)
"
                "    return s
"
                "except Exception as exc:
"
                "    print(f"[ERROR] Failed to download {ticker}: {exc}")
"
                "    return pd.Series(dtype=float)
"


In [ ]:

# ------------------------------------------------------------------
# 1️⃣ Download STOXX 600 & S&P 500
# ------------------------------------------------------------------

stoxx_px = fetch_prices_yahoo(TICKER_STOXX_600, start_date, end_date)
spx_px = fetch_prices_yahoo(TICKER_SP500, start_date, end_date)

print("STOXX 600 price series:", stoxx_px.shape, "rows")
print("S&P 500 price series:", spx_px.shape, "rows")

display(stoxx_px.to_frame().tail())
display(spx_px.to_frame().tail())


In [ ]:

            # ------------------------------------------------------------------
            # 2️⃣ Validate & clean (if processors are available)
            # ------------------------------------------------------------------

            if validator is not None and cleaner is not None:
                print("Using project validator/cleaner pipeline.\n")
"
                # Validate raw series
"
                stoxx_report = validator.validate_price_series(stoxx_px, symbol="STOXX_600")
"
                spx_report = validator.validate_price_series(spx_px, symbol="SP500")
"

"
                print("STOXX validation issues:")
"
                for issue in stoxx_report.issues:
"
                    print(" -", issue)
"

"
                print("\nS&P 500 validation issues:")
"
                for issue in spx_report.issues:
"
                    print(" -", issue)
"

"
                # Clean series
"
                stoxx_clean, stoxx_clean_report = cleaner.clean_price_series(
"
                    stoxx_px,
"
                    symbol="STOXX_600",
"
                    validation_report=stoxx_report,
"
                )
"
                spx_clean, spx_clean_report = cleaner.clean_price_series(
"
                    spx_px,
"
                    symbol="SP500",
"
                    validation_report=spx_report,
"
                )
"
            else:
"
                print("[WARN] validator/cleaner not available – using raw series as 'clean'.")
"
                stoxx_clean, spx_clean = stoxx_px.copy(), spx_px.copy()
"


In [ ]:

            # ------------------------------------------------------------------
            # 3️⃣ Compute daily returns & relative performance
            # ------------------------------------------------------------------

            def compute_returns(prices: pd.Series) -> pd.Series:
                rets = prices.pct_change().replace([np.inf, -np.inf], np.nan)
                return rets

            stoxx_ret = compute_returns(stoxx_clean).rename("STOXX_600_ret")
            spx_ret = compute_returns(spx_clean).rename("SP500_ret")

            # Align on common index
            ret_df = pd.concat([stoxx_ret, spx_ret], axis=1).dropna()

            # Relative performance = STOXX cumulative / SPX cumulative - 1
            stoxx_cum = (1 + ret_df["STOXX_600_ret"]).cumprod()
            spx_cum = (1 + ret_df["SP500_ret"]).cumprod()
            rel_perf = (stoxx_cum / spx_cum - 1).rename("EU_vs_US_relative")

            print("Returns DataFrame shape:", ret_df.shape)
"
            display(ret_df.tail())
"


In [ ]:

# ------------------------------------------------------------------
# 4️⃣ Basic risk & performance statistics
# ------------------------------------------------------------------

def summarize_returns(r: pd.Series, name: str) -> pd.Series:
    ann_factor = 252
    mean_daily = r.mean()
    vol_daily = r.std()
    ann_return = (1 + mean_daily) ** ann_factor - 1
    ann_vol = vol_daily * np.sqrt(ann_factor)
    sharpe = ann_return / ann_vol if ann_vol != 0 else np.nan
    max_dd = ((1 + r).cumprod() / (1 + r).cumprod().cummax() - 1).min()
    return pd.Series(
        {
            "mean_daily": mean_daily,
            "vol_daily": vol_daily,
            "ann_return": ann_return,
            "ann_vol": ann_vol,
            "sharpe": sharpe,
            "max_drawdown": max_dd,
        },
        name=name,
    )

stoxx_stats = summarize_returns(stoxx_ret.dropna(), "STOXX_600")
spx_stats = summarize_returns(spx_ret.dropna(), "SP500")

stats_df = pd.concat([stoxx_stats, spx_stats], axis=1)
print("📊 Risk & performance summary:")
display(stats_df.T)


In [ ]:

# ------------------------------------------------------------------
# 5️⃣ Visualizations – Indexed performance & relative performance
# ------------------------------------------------------------------

fig, axes = plt.subplots(2, 1, figsize=(12, 8), sharex=True)

# Indexed performance
base_val = 100.0
stoxx_indexed = (1 + stoxx_ret.dropna()).cumprod() * base_val
spx_indexed = (1 + spx_ret.dropna()).cumprod() * base_val

axes[0].plot(stoxx_indexed.index, stoxx_indexed, label="STOXX 600", linewidth=2)
axes[0].plot(spx_indexed.index, spx_indexed, label="S&P 500", linewidth=2)
axes[0].set_title("Indexed Performance (base 100)")
axes[0].set_ylabel("Index Level")
axes[0].legend()

# Relative performance (%)
axes[1].plot(rel_perf.index, rel_perf * 100, color="tab:green", linewidth=2)
axes[1].axhline(0, color="black", linestyle="--", linewidth=1)
axes[1].set_title("Relative Performance: Europe vs US (STOXX / S&P - 1)")
axes[1].set_ylabel("EU vs US (%)")

plt.tight_layout()
plt.show()


In [ ]:

# ------------------------------------------------------------------
# 6️⃣ Build multi-asset panel (for sectors / baskets later)
# ------------------------------------------------------------------

if aggregator is not None:
    print("Using DataAggregator to build a simple panel with STOXX and S&P.")

    series_dict = {
        "STOXX_600": stoxx_clean,
        "SP500": spx_clean,
    }
    panel, panel_report = aggregator.build_panel_from_series_dict(
        series_dict,
        name="eu_vs_us_prices",
        normalize_returns=False,
    )

    print("Panel shape:", panel.shape)
    print("Aggregation steps:")
    for step in panel_report.steps:
        print(f" - {step.action.value}: {step.description}")
else:
    print("[INFO] aggregator not available – skip panel construction for now.")


## ✅ Next Steps

You now have:

- Cleaned STOXX 600 and S&P 500 price and return series
- Basic risk and performance metrics
- A relative performance series (EU vs US)
- (Optionally) a canonical price panel via `DataAggregator`

From here you can:

- Add **macro data** (FRED / ECB) and join with market series
- Explore **sector / basket** performance by building additional panels
- Feed these series into your **signal generator** and **backtest engine**
- Validate that the input data used by the Streamlit dashboard is clean and robust

> Tip: Duplicate this notebook as `02_signal_development.ipynb` to prototype
> new momentum/value/macro signals on top of these series.